## WebQA数据集

链接: https://pan.baidu.com/s/1pLXEYtd 密码: 6fbf

文件列表：
- WebQA.v1.0/readme.txt
- WebQA.v1.0/me_test.ann.json （一个问题只配一段材料，材料中有答案）
- WebQA.v1.0/me_test.ir.json （一个问题配多段材料，材料可能有也可能没有答案）
- WebQA.v1.0/me_train.json （混合的训练语料）
- WebQA.v1.0/me_validation.ann.json （一个问题只配一段材料，材料中有答案）
- WebQA.v1.0/me_validation.ir.json （一个问题配多段材料，材料可能有也可能没有答案）

test跟validation的区别是，理论上来说，validation的分布跟train的分布更加接近。一般而言，validation用来验证模型的精确度，test用来验证模型的迁移能力。ann与ir的区别是，因为ir给每个问题配置了多段材料，可以通过各段材料投票来得到更加可靠的答案；而ann则是一问一材料的形式，是真正考验阅读理解能力的测试集。

整理后的数据格式如下，以me_train.json为例：
- 1、如果用Python的json库读取后，得到一个字典me_train，字典的键是Q_TRN_010878这样的问题标号；

- 2、通过me_train['Q_TRN_010878']获得单条记录，每条记录也是一个字典，字典下有两个键：question和evidences；

- 3、me_train['Q_TRN_010878']['question']就可以获得问题的文本内容，如“勇敢的心霍笑林的父亲是谁出演的”；

- 4、evidences是问题的材料和对应答案，也是一个字典，字典的键是Q_TRN_010878#06这样的标号；

- 5、me_train['Q_TRN_010878']['evidences']['Q_TRN_010878#05']获得单条记录，也是一个字典，字典有两个键：evidence和answer；

- 6、evidence为对应的材料，如“答：《勇敢的心》霍绍昌与华夫人的儿子杨志刚饰霍啸林简介男主角，霍家少爷，领衔主演寇振海饰霍绍昌简介霍啸林的父亲‘举人’，主演史可饰华夫人简介霍啸林和赵舒城的母亲，主演”，answer是一个答案列表（因为答案可能有多个），如[u'寇振海']，如果材料中并没有答案，那么答案是[u'no_answer']。

In [1]:
import json
import re
import smart_open
from tqdm import tqdm
import hashlib
import pandas as pd
import numpy as np

%matplotlib inline

def extract_qa_data(filepath):
    docids,questions,contexts,answers = [],[],[],[]
    
    with open(filepath, encoding="utf-8") as f:
            data_squad = json.load(f)
            for k,v in data_squad.items():
                docids.append(k)
                questions.append(v['question'])
                contexts.append([j['evidence'] for _,j in v['evidences'].items()][0])
                answers.append([j['answer'][0] for _,j in v['evidences'].items()][0])
    return docids,questions,contexts,answers

def get_df(fn):
    rows = []
    
    with  smart_open.open(fn, 'r',encoding = 'utf-8') as f:
        for line in tqdm(f):
            d = json.loads(line)
            question_id = 'dureader_' + str(d['question_id'])
            question = d['question']
            
            if len(d['answer_docs']) == 0 or len(d['fake_answers']) == 0 or d['answer_docs'][0] >= len(d['documents']):
                #print('no answer doc!')
                continue

            doc = d['documents'][d['answer_docs'][0]]
            context = doc['paragraphs'][doc['most_related_para']].replace(' ', '').replace('\t', '')
            answer = d['fake_answers'][0]
            #docid = hashlib.md5(str(answer).encode(encoding='UTF-8')).hexdigest()
            rows.append({
                'docid':question_id ,
                'context': context.replace(' ', '').replace('\t', ''),
                'question': question.replace(' ', '').replace('\t', ''),
                'answer': answer.replace(' ', '').replace('\t', '')
            })

    return pd.DataFrame(rows)



In [70]:
duread = pd.concat([
    get_df(r'E:\dureader_preprocessed\preprocessed\devset\zhidao.dev.json'),
    get_df(r'E:\dureader_preprocessed\preprocessed\devset\search.dev.json'),
    get_df(r'E:\dureader_preprocessed\preprocessed\trainset\search.train.json'),
    get_df(r'E:\dureader_preprocessed\preprocessed\trainset\zhidao.train.json'),


],
    axis=0
)

 
duread.head()

5000it [00:01, 2561.48it/s]
5000it [00:02, 2158.28it/s]
136208it [01:01, 2205.30it/s]
135366it [00:52, 2591.58it/s]


,docid,context,question,answer
0,dureader_191572,手账，指用于记事的本子。,手账是什么,记事的本子。
1,dureader_191573,<p>用交叉验证的目的是为了得到可靠稳定的模型。在建立PCR或PLS模型时，一个很重要的因素...,交叉验证的作用,为了得到可靠稳定的模型。
2,dureader_186574,嗅觉失灵是鼻部的常见疾病，病发会严重影响患者的工作生活，给患者带来了很大的负面影响。那么，鼻...,嗅觉失灵的治疗方法,（1）药物治疗：药物治疗可配合原因治疗。对于非呼吸阻塞性嗅觉失灵，尚缺乏十分有效的药物，临床...
3,dureader_186575,用鲁大师，鲁大师有一个游戏模式。,win10游戏模式怎么打开,用鲁大师，鲁大师有一个游戏模式。
4,dureader_186576,点右键按”设置“，进入“对象捕捉”选项卡，如果不是进入的这个，点击上面的“对象捕捉”就可以进...,cad捕捉不到点一直跳来跳去,点右键按”设置“，进入“对象捕捉”选项卡，如果不是进入的这个，点击上面的“对象捕捉”就可以进...


In [71]:
duread.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267812 entries, 0 to 129712
Data columns (total 4 columns):
docid       267812 non-null object
context     267812 non-null object
question    267812 non-null object
answer      267812 non-null object
dtypes: object(4)
memory usage: 10.2+ MB


In [72]:
duread.drop_duplicates(inplace = True)
duread.dropna(inplace = True)


In [73]:
duread.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267812 entries, 0 to 129712
Data columns (total 4 columns):
docid       267812 non-null object
context     267812 non-null object
question    267812 non-null object
answer      267812 non-null object
dtypes: object(4)
memory usage: 10.2+ MB


In [74]:
## 加载合并后的全部数据
df_combined = pd.read_csv(r'2021.01.10Webqa和政务训练数据合集.csv')


In [75]:
df_combined.drop_duplicates(inplace = True)
#merge_df.dropna(inplace = True)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174453 entries, 0 to 174452
Data columns (total 4 columns):
docid       174453 non-null object
context     174453 non-null object
question    174453 non-null object
answer      127059 non-null object
dtypes: object(4)
memory usage: 6.7+ MB


In [76]:
df_merge = pd.concat([df_combined,duread])

In [77]:
df_merge.head()

,docid,context,question,answer
0,Q_ANN_TST_001387,阿诺德·施瓦辛格，1947年7月30日生于奥地利，是美国好莱坞男演员、健美运动员、前美国加州...,好莱坞巨星阿诺施瓦辛格曾经是美国哪个州的州长?,加州
1,Q_ANN_TST_001672,纠正一楼的回答中国社会科学院没有院士只有学部委员我们常说的两院院士是指中国科学院和工程院的院...,我们平时说的两院院士是指中国工程院和哪个学术机构?,中国科学院
2,Q_ANN_TST_001926,格林兰岛是世界上最大的岛屿世界第一大岛——格陵兰岛，位于北美洲东北部，介于北冰洋和大西洋之间...,世界上最大的岛屿是哪个岛?,格陵兰岛
3,Q_ANN_TST_001394,最为出名的是，慈禧身边的红人（李莲英），就连红顶商人（胡雪岩）也求他为自己母亲赐过匾,清朝太后身边最著名的贴身太监叫什么?,李莲英
4,Q_ANN_TST_001663,911事件是指发生在2001年9月11日，恐怖分子胁持客机撞击美国世贸双子塔和五角大楼的事件...,美国遭受911恐怖袭击的地标性建筑叫什么?,世贸双塔


In [16]:
import pandas as pd
df_merge = pd.read_excel(r'2021.01.15 汽车主体评价问答数据集构造.xlsx','合集')

In [17]:
df_merge.head()

,docid,context,question,answer
0,S30000,天籁公爵——加长版礼宾座驾，以公爵之礼遇，开启专属之享。,哪款车在评论中的评价是中性的？,天籁公爵
1,S30001,比众泰更早一步，江淮版兰博基尼，10万可带走,哪款车在评论中的评价是中性的？,众泰
2,S30001,比众泰更早一步，江淮版兰博基尼，10万可带走,哪款车在评论中的评价是中性的？,江淮
3,S30001,比众泰更早一步，江淮版兰博基尼，10万可带走,哪款车在评论中的评价是中性的？,兰博基尼
4,S30002,近日官方公布了全[汽车]新威朗的内饰图片，威朗内饰的布局、用料以及制作工艺都有着很高的水准。,哪款车在评论中的评价是正面的？,威朗


In [18]:
def answer_index(x):
    return x.context.find(str(x.answer))



df_merge["answer_index"] = df_merge.apply(answer_index,axis=1)


In [19]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 5 columns):
docid           10321 non-null object
context         10321 non-null object
question        10321 non-null object
answer          10305 non-null object
answer_index    10321 non-null int64
dtypes: int64(1), object(4)
memory usage: 403.3+ KB


In [20]:
with open('null_docids.txt',encoding = 'utf-8') as f:
    docids = [i.strip() for i in f.readlines()]
    

In [21]:
df_merge = df_merge.sample(frac = 1.0) #打乱样本

In [22]:
df_merge.loc[~(df_merge["docid"].isin(docids)  )].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10321 entries, 5671 to 9021
Data columns (total 5 columns):
docid           10321 non-null object
context         10321 non-null object
question        10321 non-null object
answer          10305 non-null object
answer_index    10321 non-null int64
dtypes: int64(1), object(4)
memory usage: 483.8+ KB


In [23]:
df_merge  = df_merge.loc[~(df_merge["docid"].isin(docids))]


In [24]:
df_merge.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10321 entries, 5671 to 9021
Data columns (total 5 columns):
docid           10321 non-null object
context         10321 non-null object
question        10321 non-null object
answer          10305 non-null object
answer_index    10321 non-null int64
dtypes: int64(1), object(4)
memory usage: 483.8+ KB


In [25]:
df_merge.tail(100)

,docid,context,question,answer,answer_index
9029,S36655,9月20日，吉利汽车全新a+级新车帝豪gl将推向市场，这也是今年继博越、帝豪gs、远景suv...,哪款车在评论中的评价是中性的？,博越,33
4520,S33406,家用顶梁柱试驾2016款众泰z500旗舰型,哪款车在评论中的评价是中性的？,众泰z500,12
9212,S36763,活动地点：林百欣会展中心南广场c36-c41广汽丰田展位。,哪款车在评论中的评价是中性的？,广汽丰田,22
8629,S36409,7.玛莎拉蒂:改造工厂推出新车quattroporte;。,哪款车在评论中的评价是中性的？,玛莎拉蒂,2
1657,S31203,好不容易在8月再次夺得月销冠，随着新迈腾上市，以后帕萨特的日子不容易。,哪款车在评论中的评价是中性的？,迈腾,18
...,...,...,...,...,...
3314,S32457,2017款迈锐宝1.5t自动舒适版全款落地多少钱朋友们指点一下我们这里是聊城最近想订购车能优惠多少,哪款车在评论中的评价是中性的？,迈锐宝,5
6898,S35208,单拿其中雅力士的低、高配测试成绩来比对，我想大伙儿就明白了。,哪款车在评论中的评价是中性的？,雅力士,4
3272,S32423,?宝马上歌唱妹子想出名想疯了,哪款车在评论中的评价是中性的？,宝马,1
7311,S35519,东风日产京德店“金秋全关怀，安全伴你行“服务月开启啦！,哪款车在评论中的评价是中性的？,东风日产,0


In [30]:
from tqdm import tqdm
    
train_data = []

def make_train_data(df):      
    for index,row in tqdm(df.iterrows()):
        main_d = dict()
        context = row["context"].lower().replace(' ', '').replace('\t', '')
        main_d["context"] = context
        qas=[]
        inside_qas={}
        inside_qas["id"] = row ["docid"]
        inside_qas["question"]= row["question"]
        if row["answer"] is not None:
            
            if isinstance(row["answer"],float):
                #print(row["answer"])
                inside_answer=[]
                inside_qas["answers"]=inside_answer
                inside_qas["is_impossible"]=True
            else :
                inside_answer=[{"text":str(row["answer"]).lower(),"answer_start": row["answer_index"]}]
                inside_qas["answers"]=inside_answer
                inside_qas["is_impossible"]=False
            
        elif row["answer"] =='nan' or row["answer"] =='' or row["answer_index"] == -1:
            inside_answer = []
            inside_qas["answers"]=inside_answer
            inside_qas["is_impossible"]=True
            
        qas.append(inside_qas)
        main_d["qas"]=qas
        train_data.append({"title": " ","paragraphs": [main_d]})
    return train_data
          

In [31]:
train_data = make_train_data(df_merge)


10321it [00:01, 5420.70it/s]


In [33]:
train_data[246]

{'title': ' ',
 'paragraphs': [{'context': '罗孚和mg则辗转进来了中国，变成了荣威和名爵。',
   'qas': [{'id': 'S31060',
     'question': '哪款车在评论中的评价是中性的？',
     'answers': [{'text': '名爵', 'answer_start': 20}],
     'is_impossible': False}]}]}

In [34]:
train_data[9919]

{'title': ' ',
 'paragraphs': [{'context': '奥迪最新创意广告，这次主角是霸王龙......',
   'qas': [{'id': 'S34173',
     'question': '哪款车在评论中的评价是中性的？',
     'answers': [{'text': '奥迪', 'answer_start': 0}],
     'is_impossible': False}]}]}

In [35]:
store_data = {'data':train_data}

In [36]:
import json

with open(r'CAR_train.json', 'w',encoding = 'utf-8') as outfile:
    json.dump(store_data, outfile,ensure_ascii = False,indent=4)
